In [ ]:
# pull data
# select columns 
# set date as index for each df
# combine dataframes
# make new column with difference
# drop original columns
# would be helpful to model the components
# model the data
# fit and get predictions
# find mse

# write it in a function
# could write a function for each

#### Importing Data and Formatting Dataframe

In [ ]:
import pandas as pd
import 
import quandl
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARMA
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from matplotlib.pylab import rcParams

%matplotlib inline

In [ ]:
cb_iy = quandl.get("ML/USEY", authtoken="aG9j5L1aZEe7Gti-5m1D")
tyc_r = quandl.get("USTREASURY/YIELD", authtoken="aG9j5L1aZEe7Gti-5m1D")

In [ ]:
tyc_r = tyc_r[['10 YR']]
tyc_r = tyc_r.loc['1996-12-31':]

In [ ]:
cb_iy.index.names = ['Date']
tyc_r.index.names = ['Date']

In [ ]:
df = tyc_r.merge(cb_iy, right_index=True, left_index=True)

In [ ]:
df['Spread'] = df['BAMLC0A0CMEY'] - df['10 YR']

In [ ]:
df = df[['Spread']]

In [ ]:
df_weekly = df_s.resample('W')
df_weekly_mean = df_weekly.mean()

df_monthly = df_s.resample('MS')
df_monthly_mean = df_monthly.mean()

#### Plotting Data and Statistics

In [ ]:
df_list = [df, df_weekly, df_monthly]

for thing in df_list:
    thing.plot(figsize = (12, 4))

In [ ]:
for thing in df_list:
    thing.plot(figsize = (12,3), style = '.b')

In [ ]:
roll_mean = df.rolling(window=365, center=False).mean()
roll_std = df.rolling(window=365, center=False).std()

In [ ]:
# daily plot
fig = plt.figure(figsize=(12,7))
plt.plot(df, color='blue', label='Original')
plt.plot(roll_mean, color='red', label='Rolling Mean')
plt.plot(roll_std, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [ ]:
roll_mean = df_weekly_mean.rolling(window=52, center=False).mean()
roll_std = df_weekly_mean.rolling(window=52, center=False).std()

In [ ]:
# weekly plot
fig = plt.figure(figsize=(12,7))
plt.plot(df_weekly_mean, color='blue', label='Original')
plt.plot(roll_mean, color='red', label='Rolling Mean')
plt.plot(roll_std, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [ ]:
roll_mean = df_monthly_mean.rolling(window=12, center=False).mean()
roll_std = df_monthly_mean.rolling(window=12, center=False).std()

In [ ]:
# monthly plot
fig = plt.figure(figsize=(12,7))
plt.plot(df_monthly_mean, color='blue', label='Original')
plt.plot(roll_mean, color='red', label='Rolling Mean')
plt.plot(roll_std, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [ ]:
df.plot(figsize = (12, 4))
roll_mean = df.rolling(window=365, center=False).mean()
roll_std = df.rolling(window=365, center=False).std()
fig = plt.figure(figsize=(12,7))
plt.plot(df, color='blue', label='Original')
plt.plot(roll_mean, color='red', label='Rolling Mean')
plt.plot(roll_std, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

#### Dickey-Fuller Test

In [ ]:
dftest = adfuller(df)

# Extract and display test results in a user friendly manner
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dftest)

In [ ]:
print ('Results of Dickey-Fuller test: \n')

print(dfoutput)

#### Modeling

In [ ]:
# Plug the optimal parameter values into a new SARIMAX model
ARIMA_MODEL = sm.tsa.statespace.SARIMAX(df, 
                                        order=(1, 1, 1), 
                                        seasonal_order=(0, 0, 0, 2), 
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)

# Fit the model and print results
output = ARIMA_MODEL.fit()

print(output.summary().tables[1])

In [ ]:
# Call plot_diagnostics() on the results calculated above 
output.plot_diagnostics(figsize=(12, 15))
plt.show()

In [ ]:
# Get predictions starting from 01-01-2020 and calculate confidence intervals
pred = output.get_prediction(start=pd.to_datetime('2020-01-02'), dynamic=False)
pred_conf = pred.conf_int()

In [ ]:
# Plot real vs predicted values along with confidence interval

rcParams['figure.figsize'] = 12, 6

# Plot observed values
ax = df['1996':].plot(label='observed')

# Plot predicted values
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='g', alpha=0.5)

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Spread')
plt.legend()

plt.show()

#### Results

In [ ]:
# Get the real and predicted values
spread_forecasted = pred.predicted_mean
spread_truth = df['2020-01-02':]

In [ ]:
y_truth = spread_truth
y_forecasted = spread_forecasted
mean_squared_error(y_truth , y_forecasted)

#### Forecasting

In [ ]:
forecast_5= output.forecast(steps=5)
forecast_30 = output.forecast(steps=30)
forecast_60 = output.forecast(steps=60)
forecast_90 = output.forecast(steps=90)
forecast_120 = output.forecast(steps=120)

In [ ]:
# finish write-up
# non-modeling factors affecting the spread 
#   - monetary policy and fiscal policy acting in combination with each other, including quantitative easing
#   - acting in coordination with each other

